In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
print(sys.version)
print(sys.executable)

3.8.18 (default, Sep 11 2023, 13:40:15) 
[GCC 11.2.0]
/home/hd/software/anaconda3/envs/gnn/bin/python


In [3]:
!pip list | grep "torch*"

torch                         1.13.1
torch_geometric               2.4.0
torchmetrics                  1.3.0


In [4]:
from GNNInterpreter.gnninterpreter.datasets.cyclicity_dataset import CyclicityDataset
from GNNInterpreter.gnninterpreter.models.nnconv_classifier import NNConvClassifier

/home/hd/software/anaconda3/envs/gnn/lib/python3.8/site-packages/torch_geometric/typing.py:90: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/hd/software/anaconda3/envs/gnn/lib/python3.8/site-packages/torch_spline_conv/_basis_cuda.so)
  warnings.warn(


In [6]:
import torch
from tqdm.auto import trange

# Cyclicity

In [11]:
cyclicity = CyclicityDataset(seed=12345)

Using existing file rome-graphml.tgz
Extracting data/Cyclicity/raw/rome-graphml.tgz
Processing...


Loading graphs:   0%|          | 0/11534 [00:00<?, ?it/s]

Done!


In [12]:
cyclicity_train, cyclicity_val = cyclicity.train_test_split(k=10)
cyclicity_model = NNConvClassifier(node_features=len(cyclicity.NODE_CLS),
                                   edge_features=len(cyclicity.EDGE_CLS),
                                   num_classes=len(cyclicity.GRAPH_CLS),
                                   hidden_channels=32)

In [13]:
for epoch in trange(128):
    train_loss = cyclicity_train.fit_model(cyclicity_model, lr=0.001)
    train_f1 = cyclicity_train.evaluate_model(cyclicity_model)
    val_f1 = cyclicity_val.evaluate_model(cyclicity_model)
    print(f'Epoch: {epoch:03d}, '
          f'Train Loss: {train_loss:.4f}, '
          f'Train F1: {train_f1}, '
          f'Test F1: {val_f1}')

  0%|          | 0/128 [00:00<?, ?it/s]

/home/hd/software/anaconda3/envs/gnn/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 000, Train Loss: 102.5872, Train F1: {'red_cyclic': 0.4644976556301117, 'green_cyclic': 0.5760762691497803, 'acyclic': 0.5445596575737}, Test F1: {'red_cyclic': 0.43359375, 'green_cyclic': 0.5952045321464539, 'acyclic': 0.5212464332580566}
Epoch: 001, Train Loss: 15.4459, Train F1: {'red_cyclic': 0.5565381050109863, 'green_cyclic': 0.3080593943595886, 'acyclic': 0.5993878245353699}, Test F1: {'red_cyclic': 0.5679442286491394, 'green_cyclic': 0.31435078382492065, 'acyclic': 0.5935280323028564}
Epoch: 002, Train Loss: 3.8958, Train F1: {'red_cyclic': 0.550666868686676, 'green_cyclic': 0.5870422720909119, 'acyclic': 0.4957343637943268}, Test F1: {'red_cyclic': 0.5775249004364014, 'green_cyclic': 0.5839874148368835, 'acyclic': 0.49148935079574585}
Epoch: 003, Train Loss: 1.5325, Train F1: {'red_cyclic': 0.2691260874271393, 'green_cyclic': 0.5755802392959595, 'acyclic': 0.6430668830871582}, Test F1: {'red_cyclic': 0.31780821084976196, 'green_cyclic': 0.5719178318977356, 'acyclic': 0.

In [14]:
torch.save(cyclicity_model.state_dict(), 'ckpts/cyclicity.pt')

# Motif

In [2]:
from GNNInterpreter.gnninterpreter.models.gcn_classifier import GCNClassifier
from GNNInterpreter.gnninterpreter.datasets.motif_dataset import MotifDataset

/home/hd/software/anaconda3/envs/gnn/lib/python3.8/site-packages/torch_geometric/typing.py:90: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /lib64/libm.so.6: version `GLIBC_2.27' not found (required by /home/hd/software/anaconda3/envs/gnn/lib/python3.8/site-packages/torch_spline_conv/_basis_cuda.so)
  warnings.warn(


In [4]:
motif = MotifDataset(seed=12345)
motif_train, motif_val = motif.train_test_split(k=10)
motif_model = GCNClassifier(node_features=len(motif.NODE_CLS),
                            num_classes=len(motif.GRAPH_CLS),
                            hidden_channels=64,
                            num_layers=3)

Using existing file rome-graphml.tgz
Extracting data/Motif/raw/rome-graphml.tgz
Processing...


Loading graphs:   0%|          | 0/11534 [00:00<?, ?it/s]

Done!


In [7]:
for epoch in trange(128):
    train_loss = motif_train.fit_model(motif_model, lr=0.001)
    train_f1 = motif_train.evaluate_model(motif_model)
    val_f1 = motif_val.evaluate_model(motif_model)
    print(f'Epoch: {epoch:03d}, '
          f'Train Loss: {train_loss:.4f}, '
          f'Train F1: {train_f1}, '
          f'Test F1: {val_f1}')

  0%|          | 0/128 [00:00<?, ?it/s]

/home/hd/software/anaconda3/envs/gnn/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 000, Train Loss: 1.5991, Train F1: {'partial': 0.22206473350524902, 'house': 0.0, 'house_x': 0.23952768743038177, 'comp_4': 0.5253442525863647, 'comp_5': 0.45105215907096863}, Test F1: {'partial': 0.21076233685016632, 'house': 0.0, 'house_x': 0.24752475321292877, 'comp_4': 0.5132275223731995, 'comp_5': 0.42462843656539917}
Epoch: 001, Train Loss: 1.1178, Train F1: {'partial': 0.149232417345047, 'house': 0.5433225035667419, 'house_x': 0.7104277610778809, 'comp_4': 0.8746720552444458, 'comp_5': 0.8589664697647095}, Test F1: {'partial': 0.132013201713562, 'house': 0.5129683017730713, 'house_x': 0.7180384993553162, 'comp_4': 0.8633880019187927, 'comp_5': 0.858208954334259}
Epoch: 002, Train Loss: 0.5328, Train F1: {'partial': 0.521766185760498, 'house': 0.8187530040740967, 'house_x': 0.9257380366325378, 'comp_4': 0.944638192653656, 'comp_5': 0.928986132144928}, Test F1: {'partial': 0.5042017102241516, 'house': 0.7974137663841248, 'house_x': 0.9445585012435913, 'comp_4': 0.9367588758

In [8]:
torch.save(motif_model.state_dict(), 'ckpts/motif.pt')

# MUTAG

In [10]:
from GNNInterpreter.gnninterpreter.datasets.mutag_dataset import MUTAGDataset

In [12]:
mutag = MUTAGDataset(seed=12345)
mutag_train, mutag_val = mutag.train_test_split(k=10)
mutag_model = GCNClassifier(node_features=len(mutag.NODE_CLS),
                            num_classes=len(mutag.GRAPH_CLS),
                            hidden_channels=64,
                            num_layers=3)

Using existing file MUTAG.zip
Extracting data/MUTAG/raw/MUTAG.zip
Processing...
Done!


In [14]:
for epoch in trange(1024):
    train_loss = mutag_train.fit_model(mutag_model, lr=0.001)
    train_f1 = mutag_train.evaluate_model(mutag_model)
    val_f1 = mutag_val.evaluate_model(mutag_model)
    print(f'Epoch: {epoch:03d}, '
          f'Train Loss: {train_loss:.4f}, '
          f'Train F1: {train_f1}, '
          f'Test F1: {val_f1}')

  0%|          | 0/1024 [00:00<?, ?it/s]

Epoch: 000, Train Loss: 0.3795, Train F1: {'nonmutagen': 0.7572815418243408, 'mutagen': 0.8945147395133972}, Test F1: {'nonmutagen': 0.6666666865348816, 'mutagen': 0.8333333134651184}
Epoch: 001, Train Loss: 0.3698, Train F1: {'nonmutagen': 0.7735849022865295, 'mutagen': 0.8974359035491943}, Test F1: {'nonmutagen': 0.7692307829856873, 'mutagen': 0.8695651888847351}
Epoch: 002, Train Loss: 0.3617, Train F1: {'nonmutagen': 0.7346938848495483, 'mutagen': 0.8925619721412659}, Test F1: {'nonmutagen': 0.6666666865348816, 'mutagen': 0.8333333134651184}
Epoch: 003, Train Loss: 0.3936, Train F1: {'nonmutagen': 0.761904776096344, 'mutagen': 0.8936170339584351}, Test F1: {'nonmutagen': 0.6666666865348816, 'mutagen': 0.8333333134651184}
Epoch: 004, Train Loss: 0.4121, Train F1: {'nonmutagen': 0.8166666626930237, 'mutagen': 0.8999999761581421}, Test F1: {'nonmutagen': 0.7142857313156128, 'mutagen': 0.8181818127632141}
Epoch: 005, Train Loss: 0.3364, Train F1: {'nonmutagen': 0.7735849022865295, 'mut

In [15]:
torch.save(mutag_model.state_dict(), 'ckpts/mutag.pt')

# Shape

In [16]:
from GNNInterpreter.gnninterpreter.datasets.shape_dataset import ShapeDataset

In [17]:
shape = ShapeDataset(seed=12345)
shape_train, shape_val = shape.train_test_split(k=10)
shape_model = GCNClassifier(node_features=len(shape.NODE_CLS),
                            num_classes=len(shape.GRAPH_CLS),
                            hidden_channels=64,
                            num_layers=4)

Processing...


  0%|          | 0/8000 [00:00<?, ?it/s]

Done!


In [21]:
for epoch in trange(8):
    train_loss = shape_train.fit_model(shape_model, lr=0.0001)
    train_f1 = shape_train.evaluate_model(shape_model)
    val_f1 = shape_val.evaluate_model(shape_model)
    print(f'Epoch: {epoch:03d}, '
          f'Train Loss: {train_loss:.4f}, '
          f'Train F1: {train_f1}, '
          f'Test F1: {val_f1}')

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch: 000, Train Loss: 0.4083, Train F1: {'random': 0.7711340188980103, 'lollipop': 0.6439158916473389, 'wheel': 0.9667345285415649, 'grid': 0.73832106590271, 'star': 1.0}, Test F1: {'random': 0.7407407164573669, 'lollipop': 0.6603773832321167, 'wheel': 0.97826087474823, 'grid': 0.7301587462425232, 'star': 1.0}
Epoch: 001, Train Loss: 0.4055, Train F1: {'random': 0.7633368968963623, 'lollipop': 0.6521226167678833, 'wheel': 0.967391312122345, 'grid': 0.6634343266487122, 'star': 1.0}, Test F1: {'random': 0.7438596487045288, 'lollipop': 0.6702412962913513, 'wheel': 0.97826087474823, 'grid': 0.6617646813392639, 'star': 1.0}
Epoch: 002, Train Loss: 0.4043, Train F1: {'random': 0.7426955699920654, 'lollipop': 0.6068403124809265, 'wheel': 0.9531459212303162, 'grid': 0.5748662948608398, 'star': 0.9992852210998535}, Test F1: {'random': 0.712990939617157, 'lollipop': 0.6264367699623108, 'wheel': 0.9679144620895386, 'grid': 0.6040816307067871, 'star': 1.0}
Epoch: 003, Train Loss: 0.3993, Train F

In [22]:
torch.save(shape_model.state_dict(), 'ckpts/shape.pt')